## **Health & Personal Care Dataset**

# **Load the Libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

## **Loading the Dataset**

In [ ]:
meta_data = pd.read_csv("/content/amazon_metadata_health_personal_care.csv")
reviews = pd.read_csv("/content/amazon_reviews_health_personal_care.csv")

# Display first few rows
print("Metadata Dataset:")
display(meta_data.head())

print("\nReviews Dataset:")
display(reviews.head())


Metadata Dataset:


,main_category,title,average_rating,rating_number,price,store,parent_asin,date_first_available
0,Health & Personal Care,Silicone Bath Body Brush Exfoliator Shower Bac...,3.9,7,NaN,Rzoeox,B07V346GZH,"January 10, 2021"
1,Health & Personal Care,"iPhone 7 Plus 8 Plus Screen Protector, ZHXIN T...",3.8,2,NaN,ZHXIN,B075W927RH,"September 24, 2017"
2,Health & Personal Care,Zig Zag Rolling Machine 70mm Size With FREE BO...,3.9,7,NaN,NaN,B01FB26VKY,"May 27, 2016"
3,Health & Personal Care,Sting-Kill Disposable Wipes 8 Each ( Pack of 5),4.1,6,21.37,Sting-kill,B01IAI29RU,"July 11, 2016"
4,Health & Personal Care,Heated Eyelash Curler Mini Portable Electric E...,3.3,8,NaN,BiBOSS,B08CMN38RC,"July 24, 2020"



Reviews Dataset:


,rating,title,text,helpful_vote,parent_asin,verified_purchase,review_date
0,4.0,12 mg is 12 on the periodic table people! Mg f...,This review is more to clarify someone else’s ...,3,B07TDSJZMR,True,2020-02-06
1,5.0,Save the lanet using less plastic.,Love these easy multitasking bleach tablets. B...,3,B08637FWWF,True,2020-11-02
2,5.0,Fantastic,I have been suffering a couple months with hee...,0,B07KJVGNN5,True,2019-07-24
3,4.0,It holds the water and makes bubbles. That's ...,"It's cheap and it does what I wanted. The ""ma...",7,B092RP73CX,True,2022-09-04
4,1.0,Not for me,Didn't do a thing for me. Not saying they don'...,0,B08KYJLF5T,True,2022-01-20


## **Data Preprocessing**

In [ ]:
# Convert review_date with error handling
reviews["review_date"] = pd.to_datetime(reviews["review_date"], errors='coerce', infer_datetime_format=True)

# Convert date_first_available with error handling
meta_data["date_first_available"] = pd.to_datetime(meta_data["date_first_available"], errors='coerce', infer_datetime_format=True)

# Identify rows with conversion issues
print("Invalid dates in reviews:", reviews["review_date"].isna().sum())
print("Invalid dates in meta_data:", meta_data["date_first_available"].isna().sum())

# Optionally drop rows with NaT values after conversion
reviews.dropna(subset=["review_date"], inplace=True)
meta_data.dropna(subset=["date_first_available"], inplace=True)

# Merge datasets
merged_df = reviews.merge(meta_data, on="parent_asin", how="left")

print("Merged Data Preview:")
display(merged_df.head())


<ipython-input-5-6419238e334a>:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  reviews["review_date"] = pd.to_datetime(reviews["review_date"], errors='coerce', infer_datetime_format=True)
<ipython-input-5-6419238e334a>:5: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  meta_data["date_first_available"] = pd.to_datetime(meta_data["date_first_available"], errors='coerce', infer_datetime_format=True)


Invalid dates in reviews: 0
Invalid dates in meta_data: 2306
Merged Data Preview:


,rating,title_x,text,helpful_vote,parent_asin,verified_purchase,review_date,main_category,title_y,average_rating,rating_number,price,store,date_first_available
0,4.0,12 mg is 12 on the periodic table people! Mg f...,This review is more to clarify someone else’s ...,3,B07TDSJZMR,True,2020-02-06,Health & Personal Care,High Potency Magnesium Citrate Capsules 1000mg...,4.5,470.0,NaN,Life Nutrition,2019-08-11
1,5.0,Save the lanet using less plastic.,Love these easy multitasking bleach tablets. B...,3,B08637FWWF,True,2020-11-02,Health & Personal Care,Evolve Concentrated Bleach Tablets - 32-ct Bot...,4.8,181.0,NaN,Evolve,2019-09-12
2,5.0,Fantastic,I have been suffering a couple months with hee...,0,B07KJVGNN5,True,2019-07-24,Health & Personal Care,"Dr. Foot's Gel Heel Protectors, Plantar Fascii...",3.6,78.0,NaN,Dr.Foot,2018-11-15
3,4.0,It holds the water and makes bubbles. That's ...,"It's cheap and it does what I wanted. The ""ma...",7,B092RP73CX,True,2022-09-04,Health & Personal Care,"Homedics Bubble Bliss Deluxe-Foot Spa, Heat Ma...",4.4,8312.0,NaN,Homedics,2019-09-17
4,1.0,Not for me,Didn't do a thing for me. Not saying they don'...,0,B08KYJLF5T,True,2022-01-20,Health & Personal Care,Brain Supplement 1053mg - Premium Nootropic Br...,4.1,94.0,NaN,Nature's Nutrition,2020-12-08


## **DistilBERT**

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## **Data Preprocessing**

In [ ]:
# 🔹 1️⃣ Fill missing text data
reviews["text"] = reviews["text"].fillna("")

# 🔹 2️⃣ Convert ratings into sentiment labels (3+ = positive, else negative)
reviews["label"] = reviews["rating"].apply(lambda x: 1 if x >= 3 else 0)

# 🔹 3️⃣ Train-test split
X_train, X_test, y_train, y_test = train_test_split(reviews["text"], reviews["label"], test_size=0.2, random_state=42)

# 🔹 4️⃣ Load DistilBERT Tokenizer (Lighter BERT)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
max_len = 128
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=max_len, return_tensors="pt")
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=max_len, return_tensors="pt")

In [ ]:
# 🔹 6️⃣ Convert to PyTorch Dataset
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# **Splitting of Dataset**

In [ ]:
train_dataset = SentimentDataset(train_encodings, y_train.tolist())
test_dataset = SentimentDataset(test_encodings, y_test.tolist())

In [ ]:
# 🔹 7️⃣ Optimized DataLoader
batch_size = 32  # Increased batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [ ]:
# 🔹 8️⃣ Load DistilBERT Model (Lighter Version)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2).to(device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## **Loading of Adam Optimizer**

In [ ]:
# 🔹 9️⃣ Optimized Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5, betas=(0.9, 0.98), weight_decay=0.01)
loss_fn = torch.nn.CrossEntropyLoss()


In [ ]:
# 🔹 🔟 Enable Gradient Accumulation (for memory efficiency)
epochs = 3
scaler = torch.cuda.amp.GradScaler()
accumulation_steps = 4  # Accumulate gradients every 4 batches

<ipython-input-14-a99a1a73c7d3>:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


# **Training**

In [ ]:
for epoch in range(epochs):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for i, batch in enumerate(train_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with torch.cuda.amp.autocast():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss / accumulation_steps  # Scale loss

        scaler.scale(loss).backward()

        if (i + 1) % accumulation_steps == 0 or (i + 1) == len(train_loader):
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        total_loss += loss.item() * accumulation_steps

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

<ipython-input-15-d0f4844e320d>:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 1, Loss: 0.1743
Epoch 2, Loss: 0.1386
Epoch 3, Loss: 0.1164


## **Model Evaluation**

In [ ]:
# 🔹 1️⃣1️⃣ Evaluation Loop
model.eval()
preds, true_labels = [], []

In [ ]:
# 🔹 1️⃣3️⃣ Save Model Properly
model_save_path = "distilbert_sentiment"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"Model and tokenizer saved successfully at '{model_save_path}'")

Model and tokenizer saved successfully at 'distilbert_sentiment'


In [ ]:
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

In [ ]:
print("\nClassification Report:\n", classification_report(true_labels, preds, target_names=["Negative", "Positive"]))



Classification Report:
               precision    recall  f1-score   support

    Negative       0.83      0.86      0.85     25061
    Positive       0.97      0.96      0.96    100004

    accuracy                           0.94    125065
   macro avg       0.90      0.91      0.90    125065
weighted avg       0.94      0.94      0.94    125065



In [ ]:
# Calculate Accuracy
correct = sum(1 for x, y in zip(preds, true_labels) if x == y)
accuracy = correct / len(true_labels)
print(f"Test Accuracy: {accuracy:.4f}")


Test Accuracy: 0.9376
